# 0. 패키지 설치

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import shutil
#import cv2

In [ ]:
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2

# 1. 경로 설정 및 데이터 확인

In [ ]:
workspace_path = os.getcwd() 
dataset_path=os.path.join(workspace_path,'Datasets')
source_path = os.path.join(dataset_path, 'Source_code')
Wholeslide_list = []
FITC_list = []
DAPI_list = []
TRITC_list = []
for (root, directories, files) in os.walk(source_path):
    for file in files:
        temp = file.split('_')
        if temp[2] == 'Default':
            Wholeslide_list.append(os.path.join(root, file))
        elif temp[2] == 'LED-FITC':
            FITC_list.append(os.path.join(root, file))
        elif temp[2] == 'LED-DAPI':
            DAPI_list.append(os.path.join(root, file))
        elif temp[2] == 'LED-TRITC':
            TRITC_list.append(os.path.join(root, file))
        else :
            print('ERROR')

In [ ]:
Wholeslide_list = Wholeslide_list[2:]

In [ ]:
Wholeslide_list

## 2. 전처리

## 2-1. CROP & CORE EXTRACTION

In [ ]:
shutil.rmtree(os.path.join(dataset_path,'Cropped_images'))

In [ ]:
save_path=os.path.join(dataset_path,'Cropped_images')
os.makedirs(save_path,exist_ok=True)

for wholeslide_data in Wholeslide_list:
    save_target=os.path.join(save_path,'__'.join(s for s in wholeslide_data.split('/')[7:])[:-4])
    os.makedirs(save_target,exist_ok=True)
    print('__'.join(s for s in wholeslide_data.split('/')[7:])[:-4])
    img_rgb=cv2.imread(wholeslide_data, cv2.IMREAD_COLOR)
    img_crop= []
    num=0
    for height in tqdm(range(0, img_rgb.shape[0], 512)):
        for width in range(0, img_rgb.shape[1], 512):
            num += 1
            img_rgb_crop = img_rgb[height:height+512, width:width+512, :]  # 30*30 절삭(높이, 너비)
            if np.any(img_rgb_crop<210):
                img_crop.append(img_rgb_crop)
                cv2.imwrite(save_target+'/%05d.jpg' % num, img_rgb_crop)